In [2]:
from datetime import datetime as dt
import uuid 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

In [3]:
in_path = './../../../'
dataset = pd.read_csv(
    in_path+'splunk_data_180918_telenor_processed.txt',  
    encoding="ISO-8859-1", 
    dtype={
        "user_id": int, 
        "visit_id": int, 
        "sequence": int, 
        "start_time":object, 
        "event_duration":float,
        "url":str, 
        "action":str, 
        "country":str,
        "user_client":str,
        "user_client_family":str,
        "user_experience":str,
        "user_os":str,
        "apdex_user_experience":str,
        "bounce_rate":float,
        "session_duration":float
    }
)

In [4]:
t = dataset
t.columns = t.columns.str.replace('min_bedrift_event.','')
t = t[~t.action.isnull()]

# drop NaN actions or urls
t = t.dropna(axis='rows', how='any',subset=['url', 'action'])

In [5]:
len(t.index)

3605250

In [6]:
# define time variables
t['start_time'] = t['start_time'].apply(lambda x: dt.strptime(str(x), "%Y-%m-%d %H:%M:%S:%f"))
t['start_time'] = t['start_time'].apply(lambda x: x.replace(microsecond=0))

t['date'] = t['start_time'].dt.date
t['hour'] = t['start_time'].dt.hour
t['DOW'] = t['start_time'].dt.dayofweek

# create new session_id based on load = "new browser session"
# visit_id is not a good measure, since people remain logged in for 1 hour. This was previously 2 hours.
# in the App, people remain logged in for 11 months, so visit_ids could carry on for a long time
# Advice: I would define a session based on inactivity. Create new session after 30 minutes inactivity
t.sort_values(['visit_id', 'start_time'], inplace=True)

t['lag_ts'] = t.sort_values(['visit_id','start_time']).groupby('visit_id')['start_time'].shift(1)
#t['lag_ts'].fillna(t['start_time'],inplace=True) # for the first event in session
t['inactivity'] = (t['start_time'] - t['lag_ts']) / np.timedelta64(1, 'm')

In [7]:
cond_inactivity = t.inactivity > 30
cond_url_not_NaN = t.url is not np.nan
cond_lag_ts_NaN = t.lag_ts is np.nan
cond_login = ((t.url == 'https://www.telenor.no/bedrift/minbedrift/beta/#/') | (t.url == 'https://www.telenor.no/bedrift/minbedrift/beta/') | (t.url == 'https://www.telenor.no/bedrift/minbedrift/beta/mobile-app.html#/')) & ("_load_" in t.action)
cond = cond_url_not_NaN & ((cond_login & cond_lag_ts_NaN) | cond_inactivity)

t['tmp'] = cond.groupby(t.visit_id).cumsum().where(cond, 0).astype(int).replace(to_replace=0, method='ffill')

t['sequence'] = t.groupby(['tmp', 'visit_id']).cumcount() + 1
t['UUID'] = 1
t.loc[:, "UUID"] = t.groupby(['user', 'tmp', 'visit_id'])['UUID'].transform(lambda g: uuid.uuid4())

# drop all sessions with 1 event (since they are duplicates)
t['uuid_count'] = t.groupby('UUID').UUID.transform('count')
t = t[t.uuid_count > 1]

In [8]:
t = t.sort_values(by='start_time')

In [9]:
def remove(s):
    s = s[s.action_cleaned.shift() != s.action_cleaned]
    return s

t = t.groupby(['UUID']).apply(remove)

In [10]:
# drop all sessions with 1 event (since they are duplicates)
t['uuid_count'] = t.groupby('UUID').UUID.transform('count')
t = t[t.uuid_count > 1]

/home/aksel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 'UUID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  


In [12]:
with open('cleaned_dataset.p', 'wb') as f:
    pickle.dump(t, f)

In [11]:
len(t.index)

2566296

In [20]:
len(t.groupby('UUID'))

/home/aksel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'UUID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


197686

In [22]:
t.head(20)

Unnamed: 0  visit_id  sequence  \
UUID                                                                           
000063f4-a91b-44e3-bd24-f17b02fae568 1449254     1482104  25939314         1   
                                     1449253     1482102  25939314         2   
                                     1449252     1482101  25939314         3   
                                     1449251     1482100  25939314         4   
00017cda-56e3-4285-99c1-c43a38d5b8d9 2586560     2644269  28472326         1   
                                     2586561     2644270  28472326         2   
                                     2586562     2644271  28472326         3   
                                     2586563     2644272  28472326         4   
000190f2-8cde-44be-821a-4af266bfb3ef 2385114     2438933  27986231         1   
                                     2385113     2438932  27986231         2   
                                     2385112     2438931  27986231         3   
0001de31-e98a-43ce-b9bc-21744a303976 2793152     2877746  28923520         1   
                                     2793158     2877752  28923520         2   
                                     2793179     2877773  28923520         3   
                                     2793191     2877790  28923520         4   
                                     2793210     2877809  28923520         5   
                                     2793509     2878161  28923520         7   
0001f73c-3faa-47eb-8678-631a821525a7 2497210     2552763  28313385         1   
                                     2497211     2552764  28313385         2   
                                     2497212     2552765  28313385         3   

                                                      start_time  \
UUID                                                               
000063f4-a91b-44e3-bd24-f17b02fae568 1449254 2018-06-14 12:44:26   
                                     1449253 2018-06-14 12:44:37   
                                     1449252 2018-06-14 12:44:38   
                                     1449251 2018-06-14 12:44:39   
00017cda-56e3-4285-99c1-c43a38d5b8d9 2586560 2018-08-03 17:51:22   
                                     2586561 2018-08-03 17:51:23   
                                     2586562 2018-08-03 17:52:06   
                                     2586563 2018-08-03 17:52:22   
000190f2-8cde-44be-821a-4af266bfb3ef 2385114 2018-07-24 14:42:27   
                                     2385113 2018-07-24 14:42:35   
                                     2385112 2018-07-24 14:43:24   
0001de31-e98a-43ce-b9bc-21744a303976 2793152 2018-08-13 13:21:48   
                                     2793158 2018-08-13 13:21:56   
                                     2793179 2018-08-13 13:22:11   
                                     2793191 2018-08-13 13:22:23   
                                     2793210 2018-08-13 13:22:30   
                                     2793509 2018-08-13 13:25:40   
0001f73c-3faa-47eb-8678-631a821525a7 2497210 2018-07-31 14:24:17   
                                     2497211 2018-07-31 14:24:18   
                                     2497212 2018-07-31 14:24:49   

                                              event_duration  \
UUID                                                           
000063f4-a91b-44e3-bd24-f17b02fae568 1449254             NaN   
                                     1449253       10.750000   
                                     1449252        1.060000   
                                     1449251        1.120000   
00017cda-56e3-4285-99c1-c43a38d5b8d9 2586560        0.000000   
                                     2586561        1.199000   
                                     2586562       43.528000   
                                     2586563       15.970000   
000190f2-8cde-44be-821a-4af266bfb3ef 2385114        0.000000   
                                     2385113        8.101000   
                    